In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
df=pd.read_csv('data.csv')

df.head


<bound method NDFrame.head of            id diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0      842302         M       17.990         10.38          122.80     1001.0   
1      842517         M       20.570         17.77          132.90     1326.0   
2    84300903         M       19.690         21.25          130.00     1203.0   
3    84348301         M       11.420         20.38           77.58      386.1   
4    84358402         M       20.290         14.34          135.10     1297.0   
5      843786         M       12.450         15.70           82.57      477.1   
6      844359         M       18.250         19.98          119.60     1040.0   
7    84458202         M       13.710         20.83           90.20      577.9   
8      844981         M       13.000         21.82           87.50      519.8   
9    84501001         M       12.460         24.04           83.97      475.9   
10     845636         M       16.020         23.24          102.70      797.8  

In [2]:
df.drop(['Unnamed: 32','id'], axis = 1 , inplace=True)
df.columns

Index(['diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')

In [3]:
df.diagnosis.replace({"M":1,"B":0},inplace=True)
df.diagnosis.unique()

array([1, 0], dtype=int64)

In [4]:
corr = df.corr()
plt.figure(figsize=(20,20))
sns.heatmap(df.corr(), cmap='YlGnBu', annot = True)
plt.title("Correlation Map", fontweight = "bold", fontsize=16)

In [5]:
corr[abs(corr['diagnosis']) > 0.60].index

Index(['diagnosis', 'radius_mean', 'perimeter_mean', 'area_mean',
       'concavity_mean', 'concave points_mean', 'radius_worst',
       'perimeter_worst', 'area_worst', 'concavity_worst',
       'concave points_worst'],
      dtype='object')

In [6]:
import numpy as np
corr_matrix = df.corr()
threshold = 0.60 
filtre = np.abs(corr_matrix["diagnosis"]) > threshold
corr_features = corr_matrix.columns[filtre].tolist()
sns.pairplot(df[corr_features], diag_kind = "kde", markers = "+", hue = "diagnosis", palette='viridis')
plt.show()

In [7]:
y=df.iloc[:,0]
X=df.iloc[:,[1,3,4,7,8,21,23,24,27,28]]
print(y)
print(X)

0      1
1      1
2      1
3      1
4      1
5      1
6      1
7      1
8      1
9      1
10     1
11     1
12     1
13     1
14     1
15     1
16     1
17     1
18     1
19     0
20     0
21     0
22     1
23     1
24     1
25     1
26     1
27     1
28     1
29     1
      ..
539    0
540    0
541    0
542    0
543    0
544    0
545    0
546    0
547    0
548    0
549    0
550    0
551    0
552    0
553    0
554    0
555    0
556    0
557    0
558    0
559    0
560    0
561    0
562    1
563    1
564    1
565    1
566    1
567    1
568    0
Name: diagnosis, Length: 569, dtype: int64
     radius_mean  perimeter_mean  area_mean  concavity_mean  \
0         17.990          122.80     1001.0        0.300100   
1         20.570          132.90     1326.0        0.086900   
2         19.690          130.00     1203.0        0.197400   
3         11.420           77.58      386.1        0.241400   
4         20.290          135.10     1297.0        0.198000   
5         12.450           82.

In [8]:
from sklearn.neighbors import LocalOutlierFactor

# split the data to X and y before Local Outlier Factorization

y=df["diagnosis"]
X=df.drop(["diagnosis"],axis=1)
columns= df.columns.tolist()

In [9]:
lof= LocalOutlierFactor()
y_pred=lof.fit_predict(X)
y_pred[0:11]
#  1 = inlier
# -1 = outlier

array([ 1,  1,  1, -1,  1, -1,  1,  1,  1, -1,  1])

In [10]:
x_score= lof.negative_outlier_factor_
outlier_score= pd.DataFrame()
outlier_score["score"]=x_score

lofthreshold= -2.5
loffilter= outlier_score["score"]< lofthreshold
outlier_index= outlier_score[loffilter].index.tolist()


In [11]:
X= X.drop(outlier_index)
y= y.drop(outlier_index).values

In [12]:

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NeighborhoodComponentsAnalysis
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,stratify=y,random_state=42)

# Dont fit the scaler while standardizate X_test !
scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [13]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score

In [14]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix 


#X_train, X_test

knn = KNeighborsClassifier(5) # Define classifier
knn.fit(X_train, y_train) # Train model

# Make predictions
y_train_pred = knn.predict(X_train)
y_test_pred = knn.predict(X_test)

# Training set performance
knn_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
knn_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
knn_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score

# Test set performance
knn_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
knn_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
knn_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score

print('Model performance for Training set')
print('- Accuracy: %s' % knn_train_accuracy)
print('- MCC: %s' % knn_train_mcc)
print('- F1 score: %s' % knn_train_f1)
print("Confusion matrix for training set\n",confusion_matrix(y_train, y_train_pred))

print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % knn_test_accuracy)
print('- MCC: %s' % knn_test_mcc)
print('- F1 score: %s' % knn_test_f1)
print("Confusion matrix for testing set\n",confusion_matrix(y_test, y_test_pred))


Model performance for Training set
- Accuracy: 0.9748110831234257
- MCC: 0.9462780027577381
- F1 score: 0.9746571285078596
Confusion matrix for training set
 [[249   1]
 [  9 138]]
----------------------------------
Model performance for Test set
- Accuracy: 0.9649122807017544
- MCC: 0.9263533510348259
- F1 score: 0.9645279368316471
Confusion matrix for testing set
 [[107   0]
 [  6  58]]


In [15]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix 

svm_rbf = SVC(kernel='linear', C=1)
svm_rbf.fit(X_train, y_train)

# Make predictions
y_train_pred = svm_rbf.predict(X_train)
y_test_pred = svm_rbf.predict(X_test)

# Training set performance
svm_rbf_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
svm_rbf_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
svm_rbf_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score

# Test set performance
svm_rbf_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
svm_rbf_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
svm_rbf_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score

print('Model performance for Training set')
print('- Accuracy: %s' % svm_rbf_train_accuracy)
print('- MCC: %s' % svm_rbf_train_mcc)
print('- F1 score: %s' % svm_rbf_train_f1)
print("Confusion matrix for training set\n",confusion_matrix(y_train, y_train_pred))

print('----------------------------------') 
print('Model performance for Test set')
print('- Accuracy: %s' % svm_rbf_test_accuracy)
print('- MCC: %s' % svm_rbf_test_mcc)
print('- F1 score: %s' % svm_rbf_test_f1)
print("Confusion matrix for testing set\n",confusion_matrix(y_test, y_test_pred))


Model performance for Training set
- Accuracy: 0.9899244332493703
- MCC: 0.9785037567705266
- F1 score: 0.9898949289898651
Confusion matrix for training set
 [[250   0]
 [  4 143]]
----------------------------------
Model performance for Test set
- Accuracy: 0.9707602339181286
- MCC: 0.9384667634346081
- F1 score: 0.9704997170135123
Confusion matrix for testing set
 [[107   0]
 [  5  59]]


In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix 

dt = DecisionTreeClassifier(random_state=10) # Define classifier
dt.fit(X_train, y_train) # Train model

# Make predictions
y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)

# Training set performance
dt_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
dt_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
dt_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score

# Test set performance
dt_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
dt_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
dt_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score

print('Model performance for Training set')
print('- Accuracy: %s' % dt_train_accuracy)
print('- MCC: %s' % dt_train_mcc)
print('- F1 score: %s' % dt_train_f1)
print("Confusion matrix for training set\n",confusion_matrix(y_train, y_train_pred))

print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % dt_test_accuracy)
print('- MCC: %s' % dt_test_mcc)
print('- F1 score: %s' % dt_test_f1)
print("Confusion matrix for testing set\n",confusion_matrix(y_test, y_test_pred))


Model performance for Training set
- Accuracy: 1.0
- MCC: 1.0
- F1 score: 1.0
Confusion matrix for training set
 [[250   0]
 [  0 147]]
----------------------------------
Model performance for Test set
- Accuracy: 0.8888888888888888
- MCC: 0.7610661249994807
- F1 score: 0.8883236047107015
Confusion matrix for testing set
 [[99  8]
 [11 53]]


In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix 

rf = RandomForestClassifier(n_estimators=40,random_state=0) # Define classifier
rf.fit(X_train, y_train) # Train model

# Make predictions
y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)

# Training set performance
rf_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
rf_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
rf_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score

# Test set performance
rf_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
rf_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
rf_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score

print('Model performance for Training set')
print('- Accuracy: %s' % rf_train_accuracy)
print('- MCC: %s' % rf_train_mcc)
print('- F1 score: %s' % rf_train_f1)
print("Confusion matrix for training set\n",confusion_matrix(y_train, y_train_pred))

print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % rf_test_accuracy)
print('- MCC: %s' % rf_test_mcc)
print('- F1 score: %s' % rf_test_f1)
print("Confusion matrix for testing set\n",confusion_matrix(y_test, y_test_pred))


Model performance for Training set
- Accuracy: 1.0
- MCC: 1.0
- F1 score: 1.0
Confusion matrix for training set
 [[250   0]
 [  0 147]]
----------------------------------
Model performance for Test set
- Accuracy: 0.9649122807017544
- MCC: 0.9263533510348259
- F1 score: 0.9645279368316471
Confusion matrix for testing set
 [[107   0]
 [  6  58]]


In [18]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix 

mlp = MLPClassifier(alpha=1, max_iter=500,random_state=1)
mlp.fit(X_train, y_train)

# Make predictions
y_train_pred = mlp.predict(X_train)
y_test_pred = mlp.predict(X_test)

# Training set performance
mlp_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
mlp_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
mlp_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score

# Test set performance
mlp_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
mlp_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
mlp_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score

print('Model performance for Training set')
print('- Accuracy: %s' % mlp_train_accuracy)
print('- MCC: %s' % mlp_train_mcc)
print('- F1 score: %s' % mlp_train_f1)
print("Confusion matrix for training set\n",confusion_matrix(y_train, y_train_pred))

print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % mlp_test_accuracy)
print('- MCC: %s' % mlp_test_mcc)
print('- F1 score: %s' % mlp_test_f1)
print("Confusion matrix for testing set\n",confusion_matrix(y_test, y_test_pred))


Model performance for Training set
- Accuracy: 0.9924433249370277
- MCC: 0.9838577759288312
- F1 score: 0.9924269078200045
Confusion matrix for training set
 [[250   0]
 [  3 144]]
----------------------------------
Model performance for Test set
- Accuracy: 0.9707602339181286
- MCC: 0.9384667634346081
- F1 score: 0.9704997170135123
Confusion matrix for testing set
 [[107   0]
 [  5  59]]


In [19]:
from sklearn.naive_bayes import BernoulliNB

NB= BernoulliNB()
NB.fit(X_train, y_train)
y_test_pred_NB = NB.predict(X_test)

NB_test_accuracy = accuracy_score(y_test, y_test_pred_NB) # Calculate Accuracy
NB_test_mcc = matthews_corrcoef(y_test, y_test_pred_NB) # Calculate MCC
NB_test_f1 = f1_score(y_test, y_test_pred_NB, average='weighted') # Calculat
print('Model performance for Test set for gradient boosting')
print('- Accuracy: %s' % NB_test_accuracy)
print('- MCC: %s' % NB_test_mcc)
print('- F1 score: %s' % NB_test_f1)
print("Confusion matrix for testing set\n",confusion_matrix(y_test, y_test_pred_NB))



Model performance for Test set for gradient boosting
- Accuracy: 0.9473684210526315
- MCC: 0.8871974560470782
- F1 score: 0.9471006548629638
Confusion matrix for testing set
 [[104   3]
 [  6  58]]


In [20]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
#import xgboost as xgb


In [21]:
gb=GradientBoostingClassifier(random_state=20)
ab=AdaBoostClassifier()
#xgb=XGBClassifier(random_state=0,booster="gbtree")

gb.fit(X_train, y_train)
ab.fit(X_train, y_train)
#xgb.fit(X_train, y_train)

y_test_pred_gb = gb.predict(X_test)
y_test_pred_ab = ab.predict(X_test)
#y_test_pred_xgb = xgb.predict(X_test)

gb_test_accuracy = accuracy_score(y_test, y_test_pred_gb) # Calculate Accuracy
gb_test_mcc = matthews_corrcoef(y_test, y_test_pred_gb) # Calculate MCC
gb_test_f1 = f1_score(y_test, y_test_pred_gb, average='weighted') # Calculate F1-score

ab_test_accuracy = accuracy_score(y_test, y_test_pred_ab) # Calculate Accuracy
ab_test_mcc = matthews_corrcoef(y_test, y_test_pred_ab) # Calculate MCC
ab_test_f1 = f1_score(y_test, y_test_pred_ab, average='weighted') # Calculate F1-score

#xgb_test_accuracy = accuracy_score(y_test, y_test_pred_xgb) # Calculate Accuracy
#xgb_test_mcc = matthews_corrcoef(y_test, y_test_pred_xgb) # Calculate MCC
#xgb_test_f1 = f1_score(y_test, y_test_pred_xgb, average='weighted') # Calculate F1-score


print('Model performance for Test set for gradient boosting')
print('- Accuracy: %s' % gb_test_accuracy)
print('- MCC: %s' % gb_test_mcc)
print('- F1 score: %s' % gb_test_f1)
print("Confusion matrix for testing set\n",confusion_matrix(y_test, y_test_pred_gb))

print('Model performance for Test set ada boosting')
print('- Accuracy: %s' % ab_test_accuracy)
print('- MCC: %s' % ab_test_mcc)
print('- F1 score: %s' % ab_test_f1)
print("Confusion matrix for testing set\n",confusion_matrix(y_test, y_test_pred_ab))

#print('Model performance for Test set xg boosting')
#print('- Accuracy: %s' % xgb_test_accuracy)
#print('- MCC: %s' % xgb_test_mcc)
#print('- F1 score: %s' % xgb_test_f1)
#print("Confusion matrix for testing set\n",confusion_matrix(y_test, y_test_pred))


Model performance for Test set for gradient boosting
- Accuracy: 0.9590643274853801
- MCC: 0.9130465485530433
- F1 score: 0.9586996038189173
Confusion matrix for testing set
 [[106   1]
 [  6  58]]
Model performance for Test set ada boosting
- Accuracy: 0.9649122807017544
- MCC: 0.9263533510348259
- F1 score: 0.9645279368316471
Confusion matrix for testing set
 [[107   0]
 [  6  58]]


In [22]:
import time
from sklearn.naive_bayes import GaussianNB

# Define estimators
from sklearn.ensemble import StackingClassifier
#from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix 

estimator_list = [
    
    ('svm',svm_rbf),
   
    ('knn',knn),
    ('rf',rf),
    ('mlp',mlp) ]
start = time.time()


# Build stack model
stack_model = StackingClassifier(
    estimators=estimator_list,cv=5, final_estimator=GaussianNB()
)

# Train stacked model
stack_model.fit(X_train, y_train)

# Make predictions
y_train_pred = stack_model.predict(X_train)
y_test_pred = stack_model.predict(X_test)
end = time.time()
# Training set model performance
stack_model_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
stack_model_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
stack_model_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score

# Test set model performance
stack_model_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
stack_model_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
stack_model_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score

print('Model performance for Training set')
print('- Accuracy: %s' % stack_model_train_accuracy)
print('- MCC: %s' % stack_model_train_mcc)
print('- F1 score: %s' % stack_model_train_f1)
print("Confusion matrix for training set\n",confusion_matrix(y_train, y_train_pred))

print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % stack_model_test_accuracy)
print('- MCC: %s' % stack_model_test_mcc)
print('- F1 score: %s' % stack_model_test_f1)
print("Confusion matrix for testing set\n",confusion_matrix(y_test, y_test_pred))

Model performance for Training set
- Accuracy: 0.9924433249370277
- MCC: 0.983787570003184
- F1 score: 0.9924379711789703
Confusion matrix for training set
 [[249   1]
 [  2 145]]
----------------------------------
Model performance for Test set
- Accuracy: 0.9941520467836257
- MCC: 0.987552741207611
- F1 score: 0.9941426232369669
Confusion matrix for testing set
 [[107   0]
 [  1  63]]


In [23]:
print(end - start)


9.106712818145752


In [24]:
acc_test_list = {'knn':knn_test_accuracy,
'svm': svm_rbf_test_accuracy,
'dt': dt_test_accuracy,
'rf': rf_test_accuracy,
'mlp': mlp_test_accuracy,
'stack': stack_model_test_accuracy}

mcc_test_list = {'knn':knn_test_mcc,
'svm': svm_rbf_test_mcc,
'dt': dt_test_mcc,
'rf': rf_test_mcc,
'mlp': mlp_test_mcc,
'stack': stack_model_test_mcc}

f1_test_list = {'knn':knn_test_f1,
'svm': svm_rbf_test_f1,
'dt': dt_test_f1,
'rf': rf_test_f1,
'mlp': mlp_test_f1,
'stack': stack_model_test_f1}

import pandas as pd

acc_df = pd.DataFrame.from_dict(acc_test_list, orient='index', columns=['Accuracy'])
mcc_df = pd.DataFrame.from_dict(mcc_test_list, orient='index', columns=['MCC'])
f1_df = pd.DataFrame.from_dict(f1_test_list, orient='index', columns=['F1'])
df = pd.concat([acc_df, mcc_df, f1_df], axis=1)
df

In [25]:
import pandas as pd

acc_df = pd.DataFrame.from_dict(acc_test_list, orient='index', columns=['Accuracy']) 
mcc_df = pd.DataFrame.from_dict(mcc_test_list, orient='index', columns=['MCC']) 
f1_df = pd.DataFrame.from_dict(f1_test_list, orient='index', columns=['F1']) 
df = pd.concat([acc_df, mcc_df, f1_df], axis=1) 
df

,Accuracy,MCC,F1
knn,0.964912,0.926353,0.964528
svm,0.970760,0.938467,0.970500
dt,0.888889,0.761066,0.888324
rf,0.964912,0.926353,0.964528
mlp,0.970760,0.938467,0.970500
stack,0.994152,0.987553,0.994143
